In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid2_iu_10000_1/"

"../../graphs/grid2_iu_10000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.454025 seconds (365.44 k allocations: 17.986 MB, 1.72% gc time)
  0.233263 seconds (267.80 k allocations: 7.727 MB)
  3.040795 seconds (32.11 M allocations: 839.829 MB, 4.38% gc time)
Working on iteration 10 with a-norm error 15.91086768623711
Working on iteration 20 with a-norm error 10.234552504794225
Working on iteration 30 with a-norm error 5.4986291088314205
Working on iteration 40 with a-norm error 2.984257570695618
Working on iteration 50 with a-norm error 1.6124665951383594
Working on iteration 60 with a-norm error 0.8469431890903134
Working on iteration 70 with a-norm error 0.4707920577174327
Working on iteration 80 with a-norm error 0.2714523578155643
Working on iteration 90 with a-norm error 0.15397268214446397
Working on iteration 100 with a-norm error 0.08351610906152847
Working on iteration 110 with a-norm error 0.04521017954985536
Working on iteration 120 with a-norm error 0.02643208376347907
Working on iteration 130 with a-norm error 0.014235777569023712
Working on 